In [0]:
import numpy as np
import pandas as pd
import nltk
import spacy
import os
import string
import gensim

In [0]:
train = pd.read_csv("/content/train_2.csv")
train = train.dropna()
train.head()

,id,qid1,qid2,question1,question2,is_duplicate,Q1,Q2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0.0,"['step', 'step', 'guide', 'invest', 'share', '...","['step', 'step', 'guide', 'invest', 'share', '..."
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0.0,"['story', 'kohinoor', 'kohinoor', 'diamond']","['would', 'happen', 'indian', 'government', 's..."
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0.0,"['increase', 'speed', 'internet', 'connection'...","['internet', 'speed', 'increased', 'hacking', ..."
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0.0,"['mentally', 'lonely', 'solve']","['find', 'remainder', 'math2324math', 'divided..."
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0.0,"['one', 'dissolve', 'water', 'quikly', 'sugar'...","['fish', 'would', 'survive', 'salt', 'water']"


In [0]:
!wget -c http://nlp.stanford.edu/data/glove.840B.300d.zip

In [0]:
import zipfile
with zipfile.ZipFile("/content/glove.840B.300d.zip", "r") as zipread:
  zipread.extractall("/content/")
  zipread.close

In [0]:
import tensorflow as tf
import tensorflow_hub as hub

tf.logging.set_verbosity(tf.logging.ERROR)
embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/1")

In [0]:
def gse(s1, s2):
  input1 = tf.placeholder(tf.string)
  input2 = tf.placeholder(tf.string)
  encode1 = tf.nn.l2_normalize(embed(input1))
  encode2 = tf.nn.l2_normalize(embed(input2))

  scores = tf.reduce_sum(tf.multiply(encode1, encode2), axis=1)
  with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())

    [similarity] = session.run([scores], feed_dict={
        input1: [s for s in s1],
        input2: [s for s in s2]
    })
    return similarity

In [0]:
train["similarity"] = gse(train["question1"], train["question2"])

In [0]:
train.head()

,id,qid1,qid2,question1,question2,is_duplicate,Q1,Q2,similarity
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0.0,"['step', 'step', 'guide', 'invest', 'share', '...","['step', 'step', 'guide', 'invest', 'share', '...",0.000004
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0.0,"['story', 'kohinoor', 'kohinoor', 'diamond']","['would', 'happen', 'indian', 'government', 's...",0.000003
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0.0,"['increase', 'speed', 'internet', 'connection'...","['internet', 'speed', 'increased', 'hacking', ...",0.000004
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0.0,"['mentally', 'lonely', 'solve']","['find', 'remainder', 'math2324math', 'divided...",0.000001
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0.0,"['one', 'dissolve', 'water', 'quikly', 'sugar'...","['fish', 'would', 'survive', 'salt', 'water']",0.000002


In [0]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(train[["similarity"]], train["is_duplicate"], random_state=5, test_size=0.2)

import xgboost as xgb
options = {"model_param":
         {"eta": 0.15, 
         "max_depth": 4,
         "n_estimators":50,
         "sub_sample":0.5,
         "scale_pos_weight":1.75,
         "random_state":1,
         "eval_metric": "logloss",
         "objective": "binary:logistic"},
         "num_round":50,
         "test_frac":0.2}

  #training model
xgbmatrix = xgb.DMatrix(train_X, train_y)
xgb_model = xgb.train(options["model_param"], xgbmatrix, options["num_round"], verbose_eval=10)
xgb_valmatrix = xgb.DMatrix(val_X)
y_val_pred = xgb_model.predict(xgb_valmatrix)
y_val_pred #just checking

y_val_pred_binary = [1 if y>0.5 else 0 for y in y_val_pred]
from sklearn.metrics import confusion_matrix, classification_report

confusion_matrix(val_y, y_val_pred_binary)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


array([[17704, 11255],
       [ 2430, 14640]])

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(val_y, y_val_pred_binary)

0.7026874361815377